In [138]:
import unittest
import numpy as np
import importlib
import sys
sys.path.append('../')
from src import pauli_basis_class
importlib.reload(pauli_basis_class)
from src.pauli_basis_class import Pauli_Basis



In [139]:
class BasisTestCase(unittest.TestCase):
    def test_get_correct_dimensions(self):
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        result = my_pauli_basis.dimension
        self.assertEqual(result, n)

    def test_only_powers_of_2(self):
        n = 3
        with self.assertRaises(Exception) as context:
            Pauli_Basis(n)  

        self.assertEqual(str(context.exception), "Must be a power of 2")

    def test_tesnor_product_list_of_arrays(self):
        n = 2 
        my_pauli_basis = Pauli_Basis(n)

        list_1 = [np.array([[1,1,3],[3,2,1],[4,5,6]]), np.array([[4,5,7],[8,4,6],[1j,6,-6]])]
        list_2 = [np.array([[4,5,8],[8,8,9],[9,3,5]])] +  list_1
        
        result = my_pauli_basis.tensor_product_list(list_1,list_2) 

        expected_list_size = len(list_1)*len(list_2)
        expected_matrix_size = len(np.kron(list_1[0], list_2[0])) 
        self.assertEqual(expected_list_size, len(result))
        self.assertEqual(expected_matrix_size, len(result[0])) 
        self.assertTrue(np.array_equal(np.kron(list_1[-1],list_2[-1]), result[-1]))      

    def test_recursive_tensor_prod_list(self):

        n = 4
        my_pauli_basis = Pauli_Basis(n)
        list_1 = [np.array([[1,1,3],[3,2,1],[4,5,6]]), np.array([[4,5,7],[8,4,6],[1j,6,-6]])]
        list_2 = [np.array([[4,5,8],[8,8,9],[9,3,5]]), np.array([[1,2,3],[4,5,6],[7,8,9]])]
        number_of_tensor_products = 3

        result = my_pauli_basis.recursive_tensor_product_list(list_1, list_2, number_of_tensor_products)

        expected_list_size = len(list_1)*(len(list_2)**(number_of_tensor_products))
        expected_matrix_size = len(list_1[0])*len(list_2[0])**(number_of_tensor_products)

        self.assertEqual(expected_list_size, len(result))
        self.assertEqual(expected_matrix_size, len(result[0]))

    def test_pauli_basis_su_2(self):
        
        n = 2
        my_pauli_basis = Pauli_Basis(n)

        result = my_pauli_basis.pauli_basis_matrices()
        
        expected_list = [0.5*np.array([[0,1],[1,0]]), 0.5*np.array([[0,-1j],[1j, 0]]), 0.5*np.array([[0,1],[0,-1]])]
        
        self.assertTrue(np.array_equal(result[i], expected_list[i]) for i in range(len(result)))    

    
    def test_pauli_basis_works_correctly(self):

        n = 16
        my_pauli_basis = Pauli_Basis(n)

        expected_number_of_matrices = n**2 - 1
        
        output_basis_list = my_pauli_basis.pauli_basis_matrices()
        
        actual_number_of_matrices = len(output_basis_list)
    
        self.assertEqual(expected_number_of_matrices, actual_number_of_matrices)











In [140]:
unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.198s

OK


In [141]:
16*16

256